In [1]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import json
import math

import time
from selenium.webdriver.support import expected_conditions as EC


import hug



from selenium.webdriver.common.action_chains import ActionChains
responses = ["I'm sorry :( I couldn't find anything like that","I'm sorry that I can't help you", '{} is a great hotel!', '{} or {} would work!', '{} is one option, but I know others too :)']
responsesdetail = ["The hotel is {} ......","This is some information about the hotel: {}......."]
responsesrating = ["The rating is {} "]
responsesprice = ["The price is {} dollars","It costs ${} per day "]
greet = ["Hello","Hi","Hello! How can I help you"]
city = ["In which city are you looking for"]
deny = ["Ok, I'll help you search other hotels"]
af = ["Do you want to book it?","Would you like me to give you the booking website?"]
           
           
def negated_ents(phrase, ent_vals,intent):
    #ents = [e for e in ent_vals if e in phrase]
    ents = [e for e in ent_vals ]
    #print("negastes" + str(ents))
    #ends = sorted([phrase.index(e) + len(e) for e in ents])
    #start = 0
    #chunks = []
    #for end in ends:
    #    chunks.append(phrase[start:end])
    #    start = end
    result = {}
   # for chunk in chunks:
    for ent in ents:
  
        if "not" in phrase or "n't" in phrase or intent == "negate":
                    result[ent] = False
        else:
                    result[ent] = True
    return result


def interpret(message):
    data = interpreter.parse(message)
    if 'no' in message:
        data["intent"]["name"] = "deny"
    return data

#look for hotel from params
def find_hotels(params, excluded, nparams):
    #print(excluded)
    query = 'SELECT * FROM hotel'
    if len(params) > 0:
        filters = ["{}=?".format(k) for k in params] + ["{} <> ?".format(k) for k in nparams] + ["name!='?'".format(k) for k in excluded] 
        query += " WHERE " + " and ".join(filters)
    t = tuple(params.values())+tuple(nparams.values())
    
    # open connection to DB
    conn = sqlite3.connect('hoteltest1.db')
    # create a cursor
    c = conn.cursor()
    c.execute(query, t)
    return c.fetchall()

infolist = ["name","city","region","country","address","description","pricerange","rating","raterange"]

#look for extra hotel info from name
def find_info(name,info):
 # open connection to DB
    # open connection to DB
    conn = sqlite3.connect('hoteltest1.db')
    # create a cursor
    c = conn.cursor()
    index = 0
    if info in infolist:
        index = infolist.index(info)
    t = str(name)
    #print (t)
    c.execute("SELECT "+ info +" FROM hotel WHERE name=:name", {"name": t})
    row = c.fetchall()
    return row
    
        
        
# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# Create a trainer that uses this config
trainer = Trainer(config.load(r"C:\Users\Shouyan\Documents\ai\rasa_nlu-master\sample_configs\config_spacy.yml"))

# Load the training data
training_data = load_data(r'C:\Users\Shouyan\Documents\ai\train1.json')


# Create an interpreter by training the model
interpreter = trainer.train(training_data)

C:\Users\Shouyan\Anaconda3\lib\site-packages\rasa_nlu\training_data\training_data.py:176: UserWarning: Intent 'askrating' has only 1 training examples! Minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_INTENT))
C:\Users\Shouyan\Anaconda3\lib\site-packages\rasa_nlu\training_data\training_data.py:176: UserWarning: Intent 'interest3' has only 1 training examples! Minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_INTENT))
C:\Users\Shouyan\Anaconda3\lib\site-packages\rasa_nlu\training_data\training_data.py:184: UserWarning: Entity 'interest1' has only 1 training examples! minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_ENTITY))


Fitting 2 folds for each of 6 candidates, totalling 12 fits


C:\Users\Shouyan\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:105: UserWarning: Found conflicting synonym definitions for 'london'. Overwriting target 'London' with 'london'. Check your training data and remove conflicting synonym definitions to prevent this from happening.
  repr(replacement)))
C:\Users\Shouyan\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Shouyan\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Shouyan\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437

In [2]:
from selenium.webdriver.chrome.options import Options



def find_website(name):
    options = Options()
    options.headless = True
    #driver = webdriver.Chrome(CHROMEDRIVER_PATH, chrome_options=options)
    driver = webdriver.Chrome(r'C:\Users\Shouyan\Documents\ai\chromedriver_win32 (1)\chromedriver.exe',chrome_options=options) 
    driver.get('https://www.booking.com')
    name = str(name)
    driver.find_element_by_id('ss').send_keys(name)

    driver.find_element_by_class_name("sb-searchbox__button").click()

    WebDriverWait(driver, 10)


    headers = {
        "User-Agent":
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
    }


    url = driver.current_url
    #print(url)

    return url
    #return url

#soup=BeautifulSoup(html,'html.parser')

In [3]:
from itertools import chain
from random import randint
confirm = ["OK,Have a great day, the url is {}. This may take some time, and the hotel might not be exist"]


def respond(message, params, suggestions, excluded,nparams,state,names1):
    # Interpret the message
    mes = message.lower()
    


    
    if state == 0:
        parse_data = interpret(message)
        # Extract the intent
        intent = parse_data["intent"]["name"]
        #print(intent)
        # Extract the entities

            
        
        entities = parse_data["entities"]
        ent_vals = [e["value"] for e in entities]
        negated = negated_ents(message, ent_vals,intent)
     

        if intent == "greet":
            n = randint(0,2)
            state = 0
            names = "names"
            return greet[0].format(), params, suggestions, excluded, nparams,names,state,names1    
        
        """
        
        if intent != "greet" and intent != "affirm" and intent!= "deny" and not any(d.get('entity', None) == 'city' for d in entities):
            print(city[0].format())
            var = input("You: ")
            #entities["city"]["value"] = str(var)
            params["city"] = str(var)
         
         
         """   
        # Add the suggestion to the excluded list if intent is "deny"
        if intent == "deny" or "not" in message:
            excluded.extend(suggestions)
        # Fill the dictionary with entities
       
        for ent in entities:
            if ent["value"] in negated and negated[ent["value"]] != True:

               #params[ent["entity"]] = str(ent["value"])
                nparams[ent["entity"]] = str(ent["value"])

            else:
                params[ent["entity"]] = str(ent["value"])
        # Find matching hotels
        results = [
            r 
            for r in find_hotels(params, excluded,nparams) 
            if r[0] not in excluded
        ]
        # Extract the suggestions
        names = [r[0] for r in results]
        
        if intent == "greet":
            n = randint(0,3)
            state = 0
            return greet[0].format(), params, suggestions, excluded, nparams,names,state,names1    
        

        
        
        
        
        if intent == "affirm":
            
            return greet[0].format(), params, suggestions, excluded, nparams,names,state,names1
        
        
        n = min(len(results), 3)
        suggestions = names[:2]
        
        #print(len(results))
        
        if len(results) == 0:
            state = 0
            
        else:
            state = 1
            names1 = names[0]
        if "research" in mes:
            params, suggestions, excluded,nparams,names1 = {}, [], [],{},[]
            return deny[0].format(), params, suggestions, excluded, nparams,names,state,names1      
        return responses[n].format(*names), params, suggestions, excluded, nparams,names,state,names1    

            

    
    elif state == 1:
        parse_data = interpret(message)
        # Extract the intent
        intent = parse_data["intent"]["name"]
        #print(intent)
        # Extract the entities
        entities = parse_data["entities"]
        ent_vals = [e["value"] for e in entities]
        negated = negated_ents(message, ent_vals,intent)
        #print (negated)
        # Add the suggestion to the excluded list if intent is "deny"
        if intent == "deny" or "not" in message:
            excluded.extend(suggestions)
        # Fill the dictionary with entities
        if intent == "questingdetail":
            name = names1
            #print("name is" + name )
            info = list(chain.from_iterable(find_info(name,"description")))
            state = 1
            names = names1
            #print (info)
            n = randint(0, 1)
            return responsesdetail[n].format(*info), params, suggestions, excluded, nparams,names,state,names1
        
        if intent == "askrating":
            name = names1
            #print("name is" + name )
            info = list(chain.from_iterable(find_info(name,"rating")))
            state = 1
            names = names1
            #print (info)
    
            return responsesrating[0].format(*info), params, suggestions, excluded, nparams,names,state,names1
        
        if intent == "askprice":
            name = names1
            #print("name is" + name )
            info = list(chain.from_iterable(find_info(name,"price")))
            state = 1
            names = names1
            #print (info)
            n = randint(0, 1)
            return responsesprice[n].format(*info), params, suggestions, excluded, nparams,names,state,names1
        
        if intent == "affirm":
            names = names1
            #print("name is" + name )
            #info = list(chain.from_iterable(find_info(name,"price")))
            #names = names1
            #print (info)
            n = randint(0, 1)
            state = 2
            #print (state)
            return af[n].format(), params, suggestions, excluded, nparams,names,state,names1
        
        if intent == "greet":
            n = randint(0,2)
            return greet[n].format(), params, suggestions, excluded, nparams,names,state,names1
        
        
        for ent in entities:
            if ent["value"] in negated and negated[ent["value"]] != True:

               #params[ent["entity"]] = str(ent["value"])
                nparams[ent["entity"]] = str(ent["value"])

            else:
                params[ent["entity"]] = str(ent["value"])
        # Find matching hotels
        results = [
            r 
            for r in find_hotels(params, excluded,nparams) 
            if r[0] not in excluded
        ]
        # Extract the suggestions
        names = [r[0] for r in results]
        try:
            names1 = names[0]
        except:
            pass
        n = min(len(results), 3)
        suggestions = names[:2]
        if n == 0:
            state = 0
        else:
            state = 1
            
        
        return responses[n].format(*names), params, suggestions, excluded, nparams,names,state,names1
    
    
    elif state == 2:
        parse_data = interpret(message)
        # Extract the intent
        intent = parse_data["intent"]["name"]
        #print(intent)
        # Extract the entities
        entities = parse_data["entities"]
        ent_vals = [e["value"] for e in entities]
        negated = negated_ents(message, ent_vals,intent)
        # Add the suggestion to the excluded list if intent is "deny"
        msg = message.lower()
        if intent == "affirm":
            
            names = names1
            state = 0
            url = find_website(names)
            params, suggestions, excluded,nparams,names1 = {}, [], [],{},[]
            return confirm[0].format(url), params, suggestions, excluded, nparams,names,state,names1
        elif intent == "deny" or "no" in msg:
            names = names1
            state = 1
            #url = find_website(names)
            params, suggestions = {}, []
            return deny[0].format(), params, suggestions, excluded, nparams,names,state,names1
        else:
            names = names1
            state = 1
            return responses[0].format() ,params,  suggestions, excluded, nparams,names,state,names1
    
    

In [ ]:
state = 0
params, suggestions, excluded,nparams,names1 = {}, [], [],{},[]

def send(message):
    
    global params,suggestions,excluded,nparams,names1 
    global state
    
   
   
    #print("USER : {}".format(message))
    
    response, params, suggestions, excluded,nparams,names,state,names1 = respond(message, params, suggestions, excluded, nparams,state,names1)
    
    print("BOT : {}".format(response))
    return response,names
    #return response, params, suggestions, excluded,nparams,names,state,names1

# Send the messages
print("Hello, Welcome to Hotel search assistant. I've got hotel from Lodon, "+
      "Los Angles, New york and Chicago. For hotels in Chicago,"
      "I can give you some information on place of interest")

print("How can I help you today?")

time.sleep(1)

for i in range (1,50):
    
    var = input("You: ")
   # time.sleep(1)
   # try:
    send(str(var))
    #except:
      #  pass
    #time.sleep(1)





Hello, Welcome to Hotel search assistant. I've got hotel from Lodon, Los Angles, New york and Chicago. For hotels in Chicago,I can give you some information on place of interest
How can I help you today?
You: I'm looking for a place to sleep
BOT : Oakwood Aurora Midtown East or Spacious Apartment by Penn station would work!
You: In chicago
BOT : The Irving on Oakley or Modern Greek Town Apartments would work!
You: Near downtown
BOT : Gold Coast 26B or Oakwood 200 Squared would work!
You: How is it
BOT : The hotel is Gold Coast 26B is located in the Downtown Chicago district of Chicago, a 9-minute walk from Chicago Museum of Contemporary Art, 0. ......
You: Is it expensive
BOT : It costs $351 per day 
You: It is too expensive
BOT : Oakwood 200 Squared or Dewitt Hotel and Suites would work!
You: How much is it
BOT : It costs $191 per day 
You: Sounds good
BOT : Do you want to book it?
You: Yes


C:\Users\Shouyan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


BOT : OK,Have a great day, the url is https://www.booking.com/index.en-gb.html?;errorv_class_interval=1;errorv_error_url=https%253A%252F%252Fwww.booking.com%252Findex.en-gb.html%253Flabel%253Dgen173nr-1FCAEoggI46AdIM1gEaJMCiAEBmAEJuAEKyAEF2AEB6AEB-AELiAIBqAIDuALjv5PvBcACAQ%253Bsid%253Dc3ba99d55f0f602198599a17cd259d20%253Bsb_price_type%253Dtotal%2526%253B;errorv_from_sf=1;errorv_group_adults=2;errorv_label_click=undef;errorv_no_rooms=1;errorv_room1=A%252CA;errorv_sb_price_type=total;errorv_shw_aparth=1;errorv_src=index;errorv_src_elem=sb;errorv_ss=Oakwood%2520200%2520Squared;errorv_ssb=empty;errorv_top_ufis=1;label=gen173nr-1FCAEoggI46AdIM1gEaJMCiAEBmAEJuAEKyAEF2AEB6AEB-AELiAIBqAIDuALjv5PvBcACAQ;sb_price_type=total;sid=c3ba99d55f0f602198599a17cd259d20. This may take some time, and the hotel might not be exist
You: Show me great hotels in New York
BOT : Oakwood Aurora Midtown East or Spacious Apartment by Penn station would work!
You: Is it good
BOT : It costs $206 per day 
You: Can you 